In [28]:
import os

In [29]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [30]:
from pyspark.sql import SQLContext, Row
#load the historical file and convert lines to rows
lines = sc.textFile("file:///Users/anjabelcijan/Desktop/Spark/output.txt")
words = lines.map(lambda l: l.split("\t"))

#create df with columns
df = sqlContext.createDataFrame(words, ["Title", "Description", "Topic"])
df.show(5)

+--------------------+--------------------+--------+
|               Title|         Description|   Topic|
+--------------------+--------------------+--------+
|Ted Cruz’s Four W...|The Texas senator...|business|
|The case for a fi...|                    |business|
|VW board consider...|Latest blow for M...|business|
|Are We Trapped in...|The 3.9 percent u...|business|
|Tesla’s Numbers A...|Recent drama surr...|business|
+--------------------+--------------------+--------+
only showing top 5 rows



In [31]:
from pyspark.sql.types import StringType
from functools import reduce
source_df = df
print(source_df.show())

+--------------------+--------------------+-------------+
|               Title|         Description|        Topic|
+--------------------+--------------------+-------------+
|Ted Cruz’s Four W...|The Texas senator...|     business|
|The case for a fi...|                    |     business|
|VW board consider...|Latest blow for M...|     business|
|Are We Trapped in...|The 3.9 percent u...|     business|
|Tesla’s Numbers A...|Recent drama surr...|     business|
|Toyota to announc...|The company is ex...|     business|
|Trade war looms a...|US demands seem t...|     business|
|Air France-KLM bo...|                    |     business|
|Tickling Trump: W...|                    |     business|
|A Puppy Went Miss...|                    |     business|
|Crescent Point sh...|Investors rejecte...|     business|
|This MoviePass Ri...|                    |     business|
|All of the Actual...|More than half th...|entertainment|
|<em>Once Upon a T...|                    |entertainment|
|Wanna Know Yo

In [32]:
stopwords = sc.textFile("file:///Users/anjabelcijan/Desktop/spark/stopwords.txt").collect()

In [33]:
import re
punctuation = re.compile(r'[-.?!,":;()|0-9]')

In [34]:
def extract_tokens(sentence):
    """Tokenization of a given sentence.
       Drop stopwords, punctuations, numbers.
       Change the sentence to lowercase."""
    
    if(sentence) :
        tokens = punctuation.sub(' ', sentence.replace("'", " ")).lower().split()
        tokens_filtered = [word for word in tokens if (word not in stopwords) and (len(word) > 2)]
        return tokens_filtered
    else :
        return []

In [35]:
# Test your function
extract_tokens("america, World!")

['america', 'world']

In [36]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

# Create UDF
extract_tokens_udf = udf(lambda sentence: extract_tokens(sentence), ArrayType(StringType()))

# Add tokens column to DataFrame
df_tokens = source_df.withColumn('tokens', extract_tokens_udf(source_df.Description))

In [37]:
# Check if your DataFrame has the right column names and types
df_tokens.dtypes

[('Title', 'string'),
 ('Description', 'string'),
 ('Topic', 'string'),
 ('tokens', 'array<string>')]

In [39]:
print(df_tokens.show(3))

Py4JJavaError: An error occurred while calling o448.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 20.0 failed 1 times, most recent failure: Lost task 0.0 in stage 20.0 (TID 23, localhost, executor driver): java.lang.IllegalStateException: Input row doesn't have expected number of values required by the schema. 3 fields are required while 2 values are provided.
	at org.apache.spark.sql.execution.python.EvaluatePython$$anonfun$makeFromJava$15$$anonfun$apply$15.applyOrElse(EvaluatePython.scala:184)
	at org.apache.spark.sql.execution.python.EvaluatePython$.org$apache$spark$sql$execution$python$EvaluatePython$$nullSafeConvert(EvaluatePython.scala:208)
	at org.apache.spark.sql.execution.python.EvaluatePython$$anonfun$makeFromJava$15.apply(EvaluatePython.scala:180)
	at org.apache.spark.sql.SparkSession$$anonfun$6$$anonfun$apply$4.apply(SparkSession.scala:747)
	at org.apache.spark.sql.SparkSession$$anonfun$6$$anonfun$apply$4.apply(SparkSession

In [16]:
from pyspark.sql.functions import explode, desc

df_words = df_tokens.select(explode(df_tokens.tokens).alias('word'))
df_words.show()

Py4JJavaError: An error occurred while calling o279.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 1 times, most recent failure: Lost task 0.0 in stage 9.0 (TID 10, localhost, executor driver): java.lang.IllegalStateException: Input row doesn't have expected number of values required by the schema. 3 fields are required while 2 values are provided.
	at org.apache.spark.sql.execution.python.EvaluatePython$$anonfun$makeFromJava$15$$anonfun$apply$15.applyOrElse(EvaluatePython.scala:184)
	at org.apache.spark.sql.execution.python.EvaluatePython$.org$apache$spark$sql$execution$python$EvaluatePython$$nullSafeConvert(EvaluatePython.scala:208)
	at org.apache.spark.sql.execution.python.EvaluatePython$$anonfun$makeFromJava$15.apply(EvaluatePython.scala:180)
	at org.apache.spark.sql.SparkSession$$anonfun$6$$anonfun$apply$4.apply(SparkSession.scala:747)
	at org.apache.spark.sql.SparkSession$$anonfun$6$$anonfun$apply$4.apply(SparkSession.s

In [14]:
df_words.groupBy('word').count().orderBy(desc('count')).show()
#not working??

Py4JJavaError: An error occurred while calling o170.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 6.0 failed 1 times, most recent failure: Lost task 1.0 in stage 6.0 (TID 8, localhost, executor driver): java.lang.IllegalStateException: Input row doesn't have expected number of values required by the schema. 3 fields are required while 2 values are provided.
	at org.apache.spark.sql.execution.python.EvaluatePython$$anonfun$makeFromJava$15$$anonfun$apply$15.applyOrElse(EvaluatePython.scala:184)
	at org.apache.spark.sql.execution.python.EvaluatePython$.org$apache$spark$sql$execution$python$EvaluatePython$$nullSafeConvert(EvaluatePython.scala:208)
	at org.apache.spark.sql.execution.python.EvaluatePython$$anonfun$makeFromJava$15.apply(EvaluatePython.scala:180)
	at org.apache.spark.sql.SparkSession$$anonfun$6$$anonfun$apply$4.apply(SparkSession.scala:747)
	at org.apache.spark.sql.SparkSession$$anonfun$6$$anonfun$apply$4.apply(SparkSession.sc

In [15]:
from pyspark.ml.feature import NGram

bigram = NGram(n=2, inputCol='tokens', outputCol='bigrams')
df_bigram = bigram.transform(df_tokens)
print(df_bigram.show())

Py4JJavaError: An error occurred while calling o231.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8.0 (TID 9, localhost, executor driver): java.lang.IllegalStateException: Input row doesn't have expected number of values required by the schema. 3 fields are required while 2 values are provided.
	at org.apache.spark.sql.execution.python.EvaluatePython$$anonfun$makeFromJava$15$$anonfun$apply$15.applyOrElse(EvaluatePython.scala:184)
	at org.apache.spark.sql.execution.python.EvaluatePython$.org$apache$spark$sql$execution$python$EvaluatePython$$nullSafeConvert(EvaluatePython.scala:208)
	at org.apache.spark.sql.execution.python.EvaluatePython$$anonfun$makeFromJava$15.apply(EvaluatePython.scala:180)
	at org.apache.spark.sql.SparkSession$$anonfun$6$$anonfun$apply$4.apply(SparkSession.scala:747)
	at org.apache.spark.sql.SparkSession$$anonfun$6$$anonfun$apply$4.apply(SparkSession.sc

In [ ]:
# trying sth else
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
tokenizer = Tokenizer(inputCol="Title", outputCol="words")
wordsData = tokenizer.transform(df)

tokenizer = Tokenizer(inputCol="Description", outputCol="words2")
words2Data = tokenizer.transform(wordsData)

In [ ]:
hashingTF = HashingTF(inputCol="words", outputCol="raw1", numFeatures=20)
featurizedData = hashingTF.transform(words2Data)

hashingTF = HashingTF(inputCol="words2", outputCol="raw2", numFeatures=20)
featurized2Data = hashingTF.transform(featurizedData)

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
countTokens = udf(lambda words: len(words), IntegerType())
featurizedData.show(5)

In [ ]:
from pyspark.sql.types import IntegerType
remover = StopWordsRemover(inputCol="words", outputCol="filtered") 
df2 = remover.transform(featurizedData).show(truncate=False)

In [ ]:
for col_name in df2.columns:
    df2 = actual_df.withColumn(col_name, lower(col(col_name)))